In [1]:
import os
import json
import numpy as np
from pandas import DataFrame, read_csv
from typing import List
from transformers import BertModel, BertTokenizer
import torch
os.chdir("../")
from src.data.request_data import getDataTF, getDataHF
from src.data.processing_data import BERTdata


2023-03-01 10:43:08.162168: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-01 10:43:08.265336: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-01 10:43:08.268013: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/ikonkobo/anaconda3/lib/:/home/ikonkobo/anaconda3/lib/:/home/ikonkobo/anacond

In [2]:
with open(r"./inputs_data/tokens.json") as file:
    input = json.load(file)

model_id = input["model_id"]
hf_token = input["hf_token"]

In [3]:
# url_dyda_da = input["url_dyda_da"]
# df_dyda_da = getDataHF(url_dyda_da)
# getDataTF("dyda_da", "train")

In [4]:
df_dyda_da_train = read_csv("./inputs_data/data_dyda_da_train.csv", encoding="utf-8", sep="|")
df_dyda_da_train.head()

,Utterance,Dialogue_ID,Label
0,good.let ' s go now .,1,1
1,you know that is tempting but is really not go...,1,0
2,i guess you are right.but what shall we do ? i...,1,3
3,all right .,1,0
4,"say , jim , how about going for a few beers af...",1,1


In [5]:
Contexts, Labels = BERTdata(df_dyda_da_train, 5).get_contexts_labels()

In [6]:
Contexts[:3]

["that's true , but then you have to buy a record player . can you study with the radio on ? no , i listen to background music . what is the difference ? the radio has too many comerials .",
 "wow ! you're really working up a storm ! i know . i've even worked up a sweat . you look like a cooking show host--only messier . anyone home ? jen ! i'm in the kitchen ... let yourself in !",
 "honey , a typhoon is coming . jenny , come and help , we need to prepare more food . really ? wow , i don't have to go to school tomorrow . ok . dad ! i'm coming . dad , why are you taping the windows ?"]

In [7]:
Labels

array([[2, 3, 2, 3, 2],
       [2, 2, 2, 3, 2],
       [2, 1, 2, 0, 3],
       ...,
       [1, 3, 2, 2, 0],
       [2, 2, 3, 2, 3],
       [0, 1, 3, 3, 2]])

## Testing BERT embedding

In [8]:
model_name = 'bert-base-uncased'
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

input_text = "Here is some text to encode"#, "from a high school"]
# tokenizer -> token_id
input_ids = torch.tensor([tokenizer.encode(input_text, add_special_tokens=True)])

with torch.no_grad():
    # Models outputs are now tuples
    last_hidden_states = model(input_ids)[0] 
# last_hidden_states = last_hidden_states.mean(1)
print(last_hidden_states)
# size of last_hidden_states is [1,768]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[[-0.0549,  0.1053, -0.1065,  ..., -0.3551,  0.0686,  0.6506],
         [-0.5759, -0.3650, -0.1383,  ..., -0.6782,  0.2092, -0.1639],
         [-0.1641, -0.5597,  0.0150,  ..., -0.1603, -0.1346,  0.6216],
         ...,
         [ 0.2448,  0.1254,  0.1587,  ..., -0.2749, -0.1163,  0.8809],
         [ 0.0481,  0.4950, -0.2827,  ..., -0.6097, -0.1212,  0.2527],
         [ 0.9046,  0.2137, -0.5897,  ...,  0.3040, -0.6172, -0.1950]]])


In [12]:
#model = BertModel.from_pretrained("bert-base-uncased")
#embedding_matrix = model.embeddings.word_embeddings.weight

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#def embedding(texts: List[str],
#              model_id: str,
#              hf_token: str):
#    api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
#    headers = {"Authorization": f"Bearer {hf_token}"}
#    response = requests.post(api_url, 
#                             headers=headers, 
#                             json={"inputs": texts, 
#                                   "options":{"wait_for_model":True}})
#    return response.json()